# Snowflake Connection

In [1]:
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import Variant, VariantType
from snowflake.snowpark.version import VERSION

accountname = "BPSYWTN-HEXAWARE" # your accountname
username = "MOHAMMADA1" #your snowflake username
password = "Paymatix@123!" #snowflake password

connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "SYSADMIN",
    "database": "ML_MODELS",
    "schema" : "MODELS",
    "warehouse": "PAYMATIX_ML_WH"
}

def snowflake_connector():
    try:
        session = Session.builder.configs(connection_parameters).create()
        print("Connection Successfull!")
    except:
        raise ValueError("error while connecting with db")
    return session

#define a session
session = snowflake_connector()
session.telemetry_enabled = False

Connection Successfull!


In [2]:
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : CARDS_DWH_USER
Role                        : "SYSADMIN"
Database                    : "ML_MODELS"
Schema                      : "MODELS"
Warehouse                   : "SNOWPARK_WH"
Snowflake version           : 7.37.1
Snowpark for Python version : 1.8.0


In [3]:
#SNOWFLAKE

In [ ]:
churn = session.table("DATASET_CHURN").to_pandas()  ## Converts Snowflake.Table to Pandas DataFrame
print(type(churn)) 
churn.head()

# Creating Function for Stored Procedure

In [6]:
def train_churn_model(session: Session, train_table: str, train_accuracy_threshold: float = 20, 
        test_accuracy_threshold: float = 20, save_model: bool = True) -> Variant:
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import train_test_split
    from joblib import dump
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import mean_squared_error, f1_score
    import os
    import joblib

    churn = session.table(train_table).to_pandas()
    churn.dropna()
    churn.drop(columns=["CUSTOMER_ID"], inplace=True, errors='ignore')

    ## Data Preprocessing
    # Applying Label Encoding on the below columns
    le = LabelEncoder()
    churn['EDUCATION_LEVEL'] = le.fit_transform(churn['EDUCATION_LEVEL'])
    churn['MARITAL_STATUS'] = le.fit_transform(churn['MARITAL_STATUS'])
    churn['INCOME_CATEGORY'] = le.fit_transform(churn['INCOME_CATEGORY'])
    churn['CARD_CATEGORY'] = le.fit_transform(churn['CARD_CATEGORY'])
    churn['GENDER'] = le.fit_transform(churn['GENDER'])

    ## Dataset Splitting for Training and Testing
    X = churn.drop('CHURN_PREDICTION',axis=1)
    y = churn['CHURN_PREDICTION']
    X_train, X_test, y_train, y_test =  train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

    ## Model Training
    
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)

    # Make predictions using the trained model
    train_predictions = model.predict(X_train)
    train_f1_score = f1_score(y_train, train_predictions)
    test_predictions = model.predict(X_test)
    test_f1_score = f1_score(y_test, test_predictions)

    model_saved = False
    session_put = None
    if save_model:
        if train_f1_score >= train_accuracy_threshold and test_f1_score>= test_accuracy_threshold:
            # Upload the model to a stage
            import tempfile
            with tempfile.TemporaryDirectory() as model_output_dir:
            # model_output_dir = 'tmp'
                # print(model_output_dir)
                model_name = os.path.join(model_output_dir, "churn_model.joblib")
                joblib.dump(model, model_name)
                session_put = session.file.put(model_name, "@ML_MODELS.MODELS.CHURN_MODEL", overwrite=True)
                model_saved = True

    return {
        "f1 score on train": train_f1_score,
        "f1 threshold on train": train_accuracy_threshold,
        "f1 score on teet": test_f1_score,
        "f1 threshold on test": test_accuracy_threshold,
        "model saved" : model_saved,
        "Total Features" : len(X.columns),
        "Input Feartures Required" : list(X.columns),
        "session_put" : session_put
    }
    

In [7]:
train_accuracy_threshold = 0.2
test_accuracy_threshold = 0.2
save_model = True
local_train = train_churn_model(session, "DATASET_CHURN", train_accuracy_threshold, test_accuracy_threshold, save_model)
print(local_train)

{'f1 score on train': 1.0, 'f1 threshold on train': 0.2, 'f1 score on teet': 0.79320987654321, 'f1 threshold on test': 0.2, 'model saved': True, 'Total Features': 18, 'Input Feartures Required': ['CUSTOMER_AGE', 'GENDER', 'DEPENDENT_COUNT', 'EDUCATION_LEVEL', 'MARITAL_STATUS', 'INCOME_CATEGORY', 'CARD_CATEGORY', 'MONTHS_ON_BOOK', 'TOTAL_RELATIONSHIP_COUNT', 'MONTHS_INACTIVE_12_MON', 'CREDIT_LIMIT', 'TOTAL_REVOLVING_BAL', 'AVG_OPEN_TO_BUY', 'TOTAL_AMT_CHNG_Q4_Q1', 'TOTAL_TRANS_AMT', 'TOTAL_TRANS_CT', 'TOTAL_CT_CHNG_Q4_Q1', 'AVG_UTILIZATION_RATIO'], 'session_put': [PutResult(source='churn_model.joblib', target='churn_model.joblib.gz', source_size=60297, target_size=11296, source_compression='NONE', target_compression='GZIP', status='UPLOADED', message='')]}


In [8]:
#Create Stored Procedure to deploy training code in Snowflake

In [9]:
session.clear_imports()
session.clear_packages()

session.use_schema("MODELS")
session.sproc.register(
      func = train_churn_model
    , name = "CHURN_MODEL"
    , packages=["pandas", "scikit-learn", "snowflake-snowpark-python", "joblib"]
    # , return_type = StringType(), # Optional if defined in the function
    , is_permanent = True
    , replace = True
    , stage_location = '@ML_MODELS.MODELS.CHURN_MODEL'
)

The version of package 'scikit-learn' in the local environment is 1.3.1, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'joblib' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'joblib'. Your UDF might not work when the package version is different between the server and your local environment.


In [10]:
train_accuracy_threshold = 0.2
test_accuracy_threshold = 0.2
save = True

print(session.call('CHURN_MODEL', 'DATASET_CHURN', train_accuracy_threshold, test_accuracy_threshold, save))

{
  "Input Feartures Required": [
    "CUSTOMER_AGE",
    "GENDER",
    "DEPENDENT_COUNT",
    "EDUCATION_LEVEL",
    "MARITAL_STATUS",
    "INCOME_CATEGORY",
    "CARD_CATEGORY",
    "MONTHS_ON_BOOK",
    "TOTAL_RELATIONSHIP_COUNT",
    "MONTHS_INACTIVE_12_MON",
    "CREDIT_LIMIT",
    "TOTAL_REVOLVING_BAL",
    "AVG_OPEN_TO_BUY",
    "TOTAL_AMT_CHNG_Q4_Q1",
    "TOTAL_TRANS_AMT",
    "TOTAL_TRANS_CT",
    "TOTAL_CT_CHNG_Q4_Q1",
    "AVG_UTILIZATION_RATIO"
  ],
  "Total Features": 18,
  "f1 score on teet": 0.7840735068912711,
  "f1 score on train": 1,
  "f1 threshold on test": 0.2,
  "f1 threshold on train": 0.2,
  "model saved": true,
  "session_put": [
    [
      "churn_model.joblib",
      "churn_model.joblib.gz",
      60329,
      11840,
      "NONE",
      "GZIP",
      "UPLOADED",
      ""
    ]
  ]
}


In [11]:
#Create Scalar UDF for inference

In [12]:
from snowflake.snowpark.types import  FloatType, PandasSeriesType, PandasDataFrame, PandasSeries, PandasDataFrameType
from snowflake.snowpark.functions import udf
session.clear_imports()
session.clear_packages()
# Add trained model and Python packages from Snowflake Anaconda channel available on the server-side as UDF dependencies
session.add_import('@ML_MODELS.MODELS.CHURN_MODEL/churn_model.joblib')
# session.add_packages('pandas','joblib',f'scikit-learn=={sklearn.__version__}')
input_types = [FloatType(),FloatType(),FloatType(),FloatType(),FloatType(),
               FloatType(),FloatType(),FloatType(),FloatType(),FloatType(),
               FloatType(),FloatType(),FloatType(),FloatType(),FloatType(),
               FloatType(),FloatType(),FloatType()]

@udf(name='predict_churn_score', return_type=PandasSeriesType(FloatType()), session=session,replace=True,is_permanent=True,stage_location='@ML_MODELS.MODELS.CHURN_MODEL',packages=['pandas','joblib','scikit-learn'])
def predict_churn_score(df: pd.DataFrame) -> pd.Series:
    import sys
    import pandas as pd
    from joblib import load
    import sklearn

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]
    
    model_file = import_dir + 'churn_model.joblib'
    model = load(model_file)
            
    # features = ['SEARCH_ENGINE','SOCIAL_MEDIA','VIDEO','EMAIL']
    # df = pd.DataFrame([budget_allocations], columns=features)
    churn_score = model.predict(df)
    return pd.Series(churn_score)

The version of package 'joblib' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'joblib'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.1, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.


In [13]:
# Create Vectorized UDF using Batch API for inference

In [27]:
session.clear_imports()
session.clear_packages()
import sklearn
import cachetools
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
from snowflake.snowpark.functions import col, call_udf, upper, array_construct, udf

# Add trained model and Python packages from Snowflake Anaconda channel available on the server-side as UDF dependencies
session.add_import('@ML_MODELS.MODELS.CHURN_MODEL/churn_model.joblib.gz')
session.add_packages('pandas','joblib','cachetools',f'scikit-learn')

@cachetools.cached(cache={})
def load_model(filename):
    import joblib
    import sys
    import os

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
            return m

@udf(name='batch_predict_churn_score',session=session,replace=True,is_permanent=True,stage_location='@ML_MODELS.MODELS.CHURN_MODEL')
def batch_predict_roi(df: list) -> float:
    import sklearn
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder
    columns = ["CUSTOMER_AGE","GENDER","DEPENDENT_COUNT","EDUCATION_LEVEL","MARITAL_STATUS","INCOME_CATEGORY","CARD_CATEGORY","MONTHS_ON_BOOK",
    "TOTAL_RELATIONSHIP_COUNT","MONTHS_INACTIVE_12_MON","CREDIT_LIMIT","TOTAL_REVOLVING_BAL","AVG_OPEN_TO_BUY","TOTAL_AMT_CHNG_Q4_Q1",
    "TOTAL_TRANS_AMT","TOTAL_TRANS_CT","TOTAL_CT_CHNG_Q4_Q1","AVG_UTILIZATION_RATIO"]
    
    df = pd.DataFrame([df], columns=columns)

    # Applying Label Encoding on the below columns
    le = LabelEncoder()
    df['EDUCATION_LEVEL'] = le.fit_transform(df['EDUCATION_LEVEL'])
    df['MARITAL_STATUS'] = le.fit_transform(df['MARITAL_STATUS'])
    df['INCOME_CATEGORY'] = le.fit_transform(df['INCOME_CATEGORY'])
    df['CARD_CATEGORY'] = le.fit_transform(df['CARD_CATEGORY'])
    df['GENDER'] = le.fit_transform(df['GENDER'])
    
    model = load_model('churn_model.joblib.gz')
    return model.predict(df)

The version of package 'joblib' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'joblib'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'cachetools' in the local environment is 5.3.1, which does not fit the criteria for the requirement 'cachetools'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.1, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.


In [28]:
from sklearn.preprocessing import LabelEncoder
churn_data = session.table("DATASET_CHURN").to_pandas()
churn_data.dropna()
churn_data.drop(columns=["CUSTOMER_ID"], inplace=True, errors='ignore')

## Data Preprocessing
# Applying Label Encoding on the below columns
le = LabelEncoder()
churn_data['EDUCATION_LEVEL'] = le.fit_transform(churn_data['EDUCATION_LEVEL'])
churn_data['MARITAL_STATUS'] = le.fit_transform(churn_data['MARITAL_STATUS'])
churn_data['INCOME_CATEGORY'] = le.fit_transform(churn_data['INCOME_CATEGORY'])
churn_data['CARD_CATEGORY'] = le.fit_transform(churn_data['CARD_CATEGORY'])
churn_data['GENDER'] = le.fit_transform(churn_data['GENDER'])

X = churn_data.drop(columns=['CHURN_PREDICTION'], errors='ignore')
X.columns = X.columns.str.upper()

In [ ]:
test_columns = []
[test_columns.append(i.upper()) for i in list(X.columns)]
print(len(test_columns))
X[:1]


In [35]:
out = session.table('DATASET_CHURN').select(*list(X.columns), call_udf("batch_predict_churn_score", array_construct(
    col("CUSTOMER_AGE"), col("GENDER"), col("DEPENDENT_COUNT"), col("EDUCATION_LEVEL"), col("MARITAL_STATUS"), col("INCOME_CATEGORY"), col("CARD_CATEGORY"), col("MONTHS_ON_BOOK"), col("TOTAL_RELATIONSHIP_COUNT"), col("MONTHS_INACTIVE_12_MON"), col("CREDIT_LIMIT"), col("TOTAL_REVOLVING_BAL"), col("AVG_OPEN_TO_BUY"), col("TOTAL_AMT_CHNG_Q4_Q1"), col("TOTAL_TRANS_AMT"), col("TOTAL_TRANS_CT"), col("TOTAL_CT_CHNG_Q4_Q1"), col("AVG_UTILIZATION_RATIO")
    )).as_("CHURN_PREDICTION"))

In [ ]:
out.to_pandas().head()

In [ ]:
from snowflake.snowpark.functions import col, call_udf, upper, array_construct
test_df = session.createDataFrame(X)

test_df.select(*list(X.columns),call_udf("batch_predict_churn_score", array_construct(
    col("CUSTOMER_AGE"), col("GENDER"), col("DEPENDENT_COUNT"), col("EDUCATION_LEVEL"), col("MARITAL_STATUS"), col("INCOME_CATEGORY"), col("CARD_CATEGORY"), col("MONTHS_ON_BOOK"), col("TOTAL_RELATIONSHIP_COUNT"), col("MONTHS_INACTIVE_12_MON"), col("CREDIT_LIMIT"), col("TOTAL_REVOLVING_BAL"), col("AVG_OPEN_TO_BUY"), col("TOTAL_AMT_CHNG_Q4_Q1"), col("TOTAL_TRANS_AMT"), col("TOTAL_TRANS_CT"), col("TOTAL_CT_CHNG_Q4_Q1"), col("AVG_UTILIZATION_RATIO")
    )).as_("CHURN_PREDICTION")).to_pandas().head()